<a href="https://colab.research.google.com/github/andythomasc/Text-Style-Transfer/blob/main/adversarial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TODOs
###1. Write??? No. Modify files so that it's like them

###2. Write and use load_dataset

###3. Write and use training


###4. Check, write and use the model

In [1]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


# Imports

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import json
import pandas as pd
from pandas import DataFrame
import numpy as np


import sys
import matplotlib.pyplot as plt
import numpy as np
import pickle
import glob
import os

In [4]:
import time
import torchtext

from torchtext.legacy import data
import torch

#from utils import tensor2text
import torch.nn.functional as F

import math

In [5]:
from torch import nn, optim
#from tensorboardX import SummaryWriter
from torch.nn.utils import clip_grad_norm_

#from evaluator import Evaluator
#from utils import tensor2text, calc_ppl, idx2onehot, add_noise, word_drop


#Utils

Minimise

In [6]:
def tensor2text(vocab, tensor):
    tensor = tensor.cpu().numpy()
    text = []
    index2word = vocab.itos
    eos_idx = vocab.stoi['<eos>']
    unk_idx = vocab.stoi['<unk>']
    stop_idxs = [vocab.stoi['!'], vocab.stoi['.'], vocab.stoi['?']]
    for sample in tensor:
        sample_filtered = []
        prev_token = None
        for idx in list(sample):
            if prev_token in stop_idxs:
                break
            if idx == unk_idx or idx == prev_token or idx == eos_idx:
                continue
            prev_token = idx
            sample_filtered.append(index2word[idx])
            
        sample = ' '.join(sample_filtered)
        text.append(sample)

    return text

def calc_ppl(log_probs, tokens_mask):
    return (log_probs.sum() / tokens_mask.sum()).exp()

def idx2onehot(x, num_classes):
    y = x.unsqueeze(-1)
    x_onehot = torch.zeros_like(y.expand(x.size() + torch.Size((num_classes, ))))
    x_onehot.scatter_(-1, y, 1)
    return x_onehot.float()

def word_shuffle(x, l, shuffle_len):
    if not shuffle_len:
        return x

    noise = torch.rand(x.size(), dtype=torch.float).to(x.device)
    pos_idx = torch.arange(x.size(1)).unsqueeze(0).expand_as(x).to(x.device)
    pad_mask = (pos_idx >= l.unsqueeze(1)).float()

    scores = pos_idx.float() + ((1 - pad_mask) * noise + pad_mask) * shuffle_len
    x2 = x.clone()
    x2 = x2.gather(1, scores.argsort(1))

    return x2



def word_dropout_raw(x, l, unk_drop_prob, rand_drop_prob, vocab):
    if not unk_drop_prob and not rand_drop_prob:
        return x

    assert unk_drop_prob + rand_drop_prob <= 1

    noise = torch.rand(x.size(), dtype=torch.float).to(x.device)
    pos_idx = torch.arange(x.size(1)).unsqueeze(0).expand_as(x).to(x.device)
    token_mask = pos_idx < l.unsqueeze(1)

    x2 = x.clone()
    
    # drop to <unk> token
    if unk_drop_prob:
        unk_idx = vocab.stoi['<unk>']
        unk_drop_mask = (noise < unk_drop_prob) & token_mask
        x2.masked_fill_(unk_drop_mask, unk_idx)

    # drop to random_mask
    if rand_drop_prob:
        rand_drop_mask = (noise > 1 - rand_drop_prob) & token_mask
        rand_tokens = torch.randint_like(x, len(vocab))
        rand_tokens.masked_fill_(1 - rand_drop_mask, 0)
        x2.masked_fill_(rand_drop_mask, 0)
        x2 = x2 + rand_tokens
    
    return x2

def unk_dropout_(x, l, drop_prob, unk_idx):
    noise = torch.rand(x.size(), dtype=torch.float).to(x.device)
    pos_idx = torch.arange(x.size(1)).unsqueeze(0).expand_as(x).to(x.device)
    token_mask = pos_idx < l.unsqueeze(1)
    unk_drop_mask = (noise < drop_prob) & token_mask
    x.masked_fill_(unk_drop_mask, unk_idx)

def rand_dropout_(x, l, drop_prob, vocab_size):
    noise = torch.rand(x.size(), dtype=torch.float).to(x.device)
    pos_idx = torch.arange(x.size(1)).unsqueeze(0).expand_as(x).to(x.device)
    token_mask = pos_idx < l.unsqueeze(1)
    rand_drop_mask = (noise < drop_prob) & token_mask
    rand_tokens = torch.randint_like(x, vocab_size)
    rand_tokens.masked_fill_(1 - rand_drop_mask, 0)
    x.masked_fill_(rand_drop_mask, 0)
    x += rand_tokens

def word_dropout_new(x, l, unk_drop_fac, rand_drop_fac, drop_prob, vocab):
    if not unk_drop_fac and not rand_drop_fac:
        return x

    assert unk_drop_fac + rand_drop_fac <= 1

    batch_size = x.size(0)
    unk_idx = vocab.stoi['<unk>']
    unk_drop_idx = int(batch_size * unk_drop_fac)
    rand_drop_idx = int(batch_size * rand_drop_fac)

    shuffle_idx = torch.argsort(torch.rand(batch_size))
    orignal_idx = torch.argsort(shuffle_idx)

    x2 = x.clone()
    x2 = x2[shuffle_idx]
    
    if unk_drop_idx:
        unk_dropout_(x2[:unk_drop_idx], l[:unk_drop_idx], drop_prob, unk_idx)

    if rand_drop_idx:
        rand_dropout_(x2[-rand_drop_idx:], l[-rand_drop_idx:], drop_prob, len(vocab))

    x2 = x2[orignal_idx]

    return x2

def word_dropout(x, l, drop_prob, unk_idx):
    if not drop_prob:
        return x

    noise = torch.rand(x.size(), dtype=torch.float).to(x.device)
    pos_idx = torch.arange(x.size(1)).unsqueeze(0).expand_as(x).to(x.device)
    token_mask = pos_idx < l.unsqueeze(1)

    drop_mask = (noise < drop_prob) & token_mask
    x2 = x.clone()
    x2.masked_fill_(drop_mask, unk_idx)
    
    return x2

def word_drop(x, l, drop_prob, pad_idx):
    if not drop_prob:
        return x

    noise = torch.rand(x.size(), dtype=torch.float).to(x.device)
    pos_idx = torch.arange(x.size(1)).unsqueeze(0).expand_as(x).to(x.device)
    token_mask = pos_idx < (l.unsqueeze(1) - 1)

    drop_mask = (noise < drop_prob) & token_mask
    x2 = x.clone()
    pos_idx.masked_fill_(drop_mask, x.size(1) - 1)
    pos_idx = torch.sort(pos_idx, 1)[0]
    x2 = x2.gather(1, pos_idx)
    
    return x2

def add_noise(words, lengths, shuffle_len, drop_prob, unk_idx):
    words = word_shuffle(words, lengths, shuffle_len)
    words = word_dropout(words, lengths, drop_prob, unk_idx)
    return words 

# Load the data

check the main as well??


In [7]:
class DatasetIterator(object):
  def __init__(self, pos_iter, neg_iter):
    self.pos_iter = pos_iter
    self.neg_iter = neg_iter

  def __iter__(self):
    for batch_pos, batch_neg in zip(iter(self.pos_iter), iter(self.neg_iter)):
      if batch_pos.text.size(0) == batch_neg.text.size(0):
        yield batch_pos.text, batch_neg.text

In [8]:
#paths
#train_pos_path = '/content/drive/My Drive/Colab Notebooks/style_transfer/training_files/label_1.txt'
#train_neg_path = '/content/drive/My Drive/Colab Notebooks/style_transfer/training_files/label_0.txt'

In [9]:


def load_dataset(config, train_pos='label_1.txt',train_neg='label_0.txt',
                 dev_pos = 'label_1.txt', dev_neg = 'label_0.txt',
                 test_pos = 'label_1.txt', test_neg = 'label_0.txt'):
  
  #Go the the root path and then make use of the parsed strings
  #eg train.pos 
  root = config.data_path

  #inspect this??
  TEXT = data.Field(batch_first= True, eos_token='<eos>')


##building dataset
    #lambda function easy for map
  dataset_fn = lambda name: data.TabularDataset(
      path = root + name,
      format = 'tsv',
      fields = [('text', TEXT)]
  )
  train_pos_set, train_neg_set = map(dataset_fn, [train_pos, train_neg])  #train dataset
  dev_pos_set, dev_neg_set = map(dataset_fn, [dev_pos, dev_neg])          #dev dataset
  test_pos_set, test_neg_set = map(dataset_fn, [test_pos, test_neg])      #test dataset

  #vocab part
  TEXT.build_vocab(train_pos_set, train_neg_set, min_freq=config.min_freq)
  
  #using GloVe from stanford for pretrained embeddings
  if config.load_pretrained_embed:
    start = time.time()

    vectors = torchtext.vocab.GloVe('6B',dim=config.embed_size, cache=config.pretrained_embed_path)
    TEXT.vocab.set_vectors(vectors.stoi, vectors.vectors, vectors.dim)
    print('vectors', TEXT.vocab.vectors.size())

    print('load embedding took {:.2f} s.'.format(time.time() - start))


  vocab = TEXT.vocab


  dataiter_fn = lambda dataset, train: data.BucketIterator(
      dataset = dataset,
      batch_size = config.batch_size,
      shuffle=train,
      repeat=train,
      sort_key = lambda x: len(x.test),
      sort_within_batch=False,
      device=config.device
  )

  train_pos_iter, train_neg_iter = map(lambda x: dataiter_fn(x, True), [train_pos_set, train_neg_set])
  dev_pos_iter, dev_neg_iter = map(lambda x: dataiter_fn(x, False), [dev_pos_set, dev_neg_set])
  test_pos_iter, test_neg_iter = map(lambda x: dataiter_fn(x, False), [test_pos_set,test_neg_set])

  train_iters = DatasetIterator(train_pos_iter, train_neg_iter)
  dev_iters = DatasetIterator(dev_pos_iter, dev_neg_iter)
  test_iters = DatasetIterator(test_pos_iter, test_neg_iter)

  return train_iters, dev_iters, test_iters, vocab





In [10]:

class Config():
    data_path = '/content/drive/My Drive/Colab Notebooks/style_transfer/training_files/'
    log_dir = '/content/drive/My Drive/Colab Notebooks/style_transfer/training_files/runs/exp'
    save_path = '/content/drive/My Drive/Colab Notebooks/style_transfer/training_files//save'
    pretrained_embed_path = '/content/drive/My Drive/Colab Notebooks/style_transfer/embedding/'
    device = torch.device('cuda' if True and torch.cuda.is_available() else 'cpu')
    discriminator_method = 'Multi' # 'Multi' or 'Cond'
    load_pretrained_embed = False
    min_freq = 3
    max_length = 400  # modified from 16
    embed_size = 256
    d_model = 256
    h = 4
    num_styles = 2
    num_classes = num_styles + 1 if discriminator_method == 'Multi' else 2
    num_layers = 4
    batch_size = 1#10
    lr_F = 0.0001
    lr_D = 0.0001
    L2 = 0
    iter_D = 10
    iter_F = 5
    F_pretrain_iter = 10#500
    log_steps = 5
    eval_steps = 25
    learned_pos_embed = True
    dropout = 0
    drop_rate_config = [(1, 0)]
    temperature_config = [(1, 0)]

    slf_factor = 0.25
    cyc_factor = 0.5
    adv_factor = 1

    inp_shuffle_len = 0
    inp_unk_drop_fac = 0
    inp_rand_drop_fac = 0
    inp_drop_prob = 0



In [11]:
config = Config()

In [12]:
#train_iters, dev_iters, test_iters, vocab = load_dataset(config)

In [13]:
#train_iters

# Methods

In [14]:
def get_lengths(tokens, eos_idx):
  lengths = torch.cumsum(tokens == eos_idx, 1)
  lengths = (lengths == 0).long().sum(-1)
  lengths = lengths + 1 # +1 for <eos> token
  return lengths

In [15]:
def batch_preprocess(batch, pad_idx, eos_idx, reverse = False):
  batch_pos, batch_neg = batch  #returning the positive and negative to containers
  diff = batch_pos.size(1) - batch_neg.size(1) # filling them with pads if they're distinct in size

  if diff < 0: 
    pad = torch.full_like(batch_neg[:, :-diff], pad_idx)
    batch_pos = torch.cat((batch_pos, pad), 1)

  elif diff > 0:
    pad = torch.full_like(batch_pos[:, :diff], pad_idx)
    batch_neg = torch.cat((batch_neg, pad), 1)

  pos_styles = torch.ones_like(batch_pos[:, 0])
  neg_styles = torch.zeros_like(batch_neg[:, 0])


  if reverse:
    batch_pos, batch_neg = batch_neg, batch_pos
    pos_styles, neg_styles = neg_styles, pos_stlyes

  tokens = torch.cat((batch_pos, batch_neg), 0)
  lengths = get_lengths(tokens, eos_idx)
  styles = torch.cat((pos_styles, neg_styles), 0)


  return tokens, lengths, styles

  


In [16]:
def d_step(config, vocab, model_F, model_D, optimizer_D, batch, temperature):
  model_F.eval()
  pad_idx = vocab.stoi['<pad>']
  eos_idx = vocab.stoi['<eos>']
  vocab_size = len(vocab)
  loss_fn = nn.NLLLoss(reduction = 'none')


  inp_tokens, inp_lengths, raw_styles = batch_preprocess(batch, pad_idx, eos_idx)
  rev_styles = 1 - raw_styles
  batch_size = inp_tokens.size(0)
  with torch.no_grad():
    raw_gen_log_probs = model_F(
        inp_tokens,
        None,
        inp_lengths,
        raw_styles,
        generate = True,
        differentiable_decode = True,
        temperature = temperature,
    )
    rev_gen_log_probs = model_F(
        inp_tokens,
        None,
        inp_lengths,
        rev_styles,
        generate = True,
        differentiable_decode = True,
        temperature = temperature,
    )


    raw_gen_soft_tokens = raw_gen_log_probs.exp()
    raw_gen_lengths = get_lengths(raw_gen_soft_tokens.argmax(-1), eos_idx)

    rev_gen_soft_tokens = rev_gen_log_probs.exp()
    rev_gen_lengths = get_lengths(rev_gen_soft_tokens.argmax(-1), eos_idx)




    if config.discriminator_method == 'Multi':
      gold_log_probs = model_D(inp_tokens, inp_lengths)
      gold_labels = raw_styles + 1

      raw_gen_log_probs = model_D(raw_gen_soft_tokens, raw_gen_lengths)
      rev_gen_log_probs = model_D(rev_gen_soft_tokens, rev_gen_lengths)
      gen_log_probs = torch.cat((raw_gen_log_probs, rev_gen_log_probs), 0)
      raw_gen_labels = raw_styles + 1
      rev_gen_labels = torch.zeros_like(rev_styles)
      gen_labels = torch.cat((raw_gen_labels, rev_gen_labels), 0)

    else:
      raw_gold_log_probs = model_D(inp_tokens, inp_lengths, raw_styles)
      rev_gold_log_probs = model_D(inp_tokens, inp_lengths, rev_styles)
      gold_lob_probs = torch.cat((raw_gold_log_probs, rev_gold_log_probs), 0)
      raw_gold_labels = torch.ones_like(raw_styles)
      rev_gold_labels = torch.zeros_like(rev_styles)
      gold_labels = torch.cat((raw_gold_labels, rev_gold_labels), 0)


      raw_gen_log_probs = model_D(raw_gen_soft_tokens, raw_gen_lengths, raw_styles)
      rev_gen_log_probs = model_D(rev_gen_soft_tokens, rev_gen_lengths, rev_styles)
      gen_log_probs = torch.cat((raw_gen_log_probs, rev_gen_log_probs), 0)
      raw_gen_labels = torch.ones_like(raw_styles)
      rev_gen_labels = torch.zeros_like(rev_syles)
      gen_labels = torch.cat((raw_gen_labels, rev_gen_labels), 0)


    adv_log_probs = torch.cat((gold_log_probs, gen_log_probs), 0)
    adv_labels = torch.cat((gold_labels, gen_labels), 0)
    adv_loss = loss_fn(adv_log_probs, adv_labels)
    assert len(adv_loss.size()) == 1
    adv_loss = adv_loss.sum() / batch_size
    loss = adv_loss 

    optimizer_D.zero_grad()
    loss.requires_grad = True
    loss.backward()

    clip_grad_norm_(model_D.parameters(), 5)
    optimizer_D.step()

    model_F.train()


    return adv_loss.item()
      

    




In [17]:
def f_step(config, vocab, model_F, model_D, optimizer_F, batch, temperature, drop_decay, cyc_rec_enable=True):
  model_D.eval()
  pad_idx = vocab.stoi['<pad>']
  eos_idx = vocab.stoi['<eos>']
  unk_idx = vocab.stoi['<unk']
  vocab_size = len(vocab)
  loss_fn = nn.NLLLoss(reduction = 'none')


  inp_tokens, inp_lengths, raw_styles = batch_preprocess(batch, pad_idx, eos_idx)
  rev_styles = 1 - raw_styles
  batch_size = inp_tokens.size(0)
  token_mask = (inp_tokens != pad_idx).float()



  optimizer_F.zero_grad()



  #se,f reconstruction loss???



  noise_inp_tokens = word_drop(
      inp_tokens,
      inp_lengths,
      config.inp_drop_prob * drop_decay,
      vocab
  )

  noise_inp_lengths = get_lengths(noise_inp_tokens, eos_idx)

  slf_log_probs = model_F(
      noise_inp_tokens,
      inp_tokens,
      noise_inp_lengths,
      raw_styles,
      generate = False,
      differentiable_decode = False,
      temperature = temperature,
  )

  slf_rec_loss = loss_fn(slf_log_probs.transpose(1, 2), inp_tokens) * token_mask
  slf_rec_loss = slf_rec_loss.sum() / batch_size
  slf_rec_loss *= config.slf_factor

  slf_rec_loss.backward()


  #cycle consistency loss

  if not cyc_rec_enable:
    optimizer_F.step()
    model_D.train()
    return slf_rec_loss.item(), 0, 0

  
  gen_log_probs = model_F(
      inp_tokens,
      None,
      inp_lengths,
      rev_styles,
      generate=True,
      differentiable_decode = True,
      temperature = temperature,
  )

  gen_soft_tokens = gen_log_probs.exp()
  gen_lengths = get_lengths(gen_soft_tokens.argmax(-1), eos_idx)


  cyc_log_probs = model_F(
      gen_soft_tokens,
      inp_tokens,
      gen_lengths,
      raw_styles,
      generate=False,
      differentiable_decode=False,
      temperature=temperature,
  )

  cyc_rec_loss = loss_fn(cyc_log_probs.transpose(1,2), inp_tokens) * token_mask
  cyc_rec_loss = cyc_rec_loss.sum() / batch_size
  cyc_rec_loss *= config.cyc_factor


  #style consistency loss

  adv_log_probs = model_D(gen_soft_tokens, gen_lengths, rev_styles)
  if config.discriminator_method == 'Multi':
    adv_labels = rev_styles + 1

  else:
    adv_labels = torch.ones_like(rev_styles)


  adv_loss = loss_fn(adv_log_probs, adv_labels)
  adv_loss = adv_loss.sum() / batch_size
  adv_loss *= config.adv_factor


  (cyc_rec_loss + adv_loss).backward()

  #update parameters

  clip_grad_norm_(model_F.parameters(), 5)
  optimizer_F.step()

  model_D.train()

  return slf_rec_loss.item(), cyc_rec_loss.item(), adv_loss.item()







In [18]:
def train(config, vocab, model_F, model_D, train_iters, dev_iters, test_iters):
  optimizer_F = optim.Adam(model_F.parameters(), lr=config.lr_F, weight_decay=config.L2)
  optimizer_D = optim.Adam(model_D.parameters(), lr=config.lr_D, weight_decay=config.L2)

  #declaring history list for losses
  his_d_adv_loss=[]
  his_f_slf_loss=[]
  his_f_cyc_loss=[]
  his_f_adv_loss=[]

  #writer = SummaryWriter(config.lor_dir)
  global_step=0
  model_F.train()
  model_D.train()


  config.save_folder = config.save_path + '/' + str(time.strftime('%b%d%H%M%S', time.localtime()))
  os.makedirs(config.save_folder)
  os.makedirs(config.save_folder + '/ckpts')
  print('Save Path:', config.save_folder)


  print('Model F pretraining......')
  for i, batch in enumerate(train_iters):
    if i>= config.F_pretrain_iter:
      break

    slf_loss, cyc_loss, _ = f_step(config, vocab, model_F, model_D, optimizer_F, batch, 1.0, 1.0, False)
    his_f_slf_loss.append(slf_loss)
    his_f_cyc_loss.append(cyc_loss)

    if (i + 1) % 10 == 0:
      avg_f_slf_loss = np.mean(his_f_slf_loss)
      avg_f_cyc_loss = np.mean(his_f_cyc_loss)
      his_f_slf_loss = []
      his_f_cyc_loss = []
      print('[iter: {}] slf_loss:{:.4f}, rec_loss:{:.4f}'.format(i + 1, avg_f_slf_loss, avg_f_cyc_loss))


  def calc_temperature(temperature_config, step):
    num = len(temperature_config)
    for i in range(num):
      t_a, s_a = temperature_config[i]
      if i == num - 1:
        return t_a

      t_b, s_b = temperature_config[i+1]
      if s_a <= step < s_b:
        k = (step - s_a) / (s_b - s_a)
        temperature = (1 - k) * t_a + k * t_b
        return temperature


  batch_iters = iter(train_iters)
  while True:
    drop_decay = calc_temperature(config.drop_rate_config, global_step)
    temperature = calc_temperature(config.temperature_config, global_step)
    batch = next(batch_iters)

    for _ in range(config.iter_D):
      batch = next(batch_iters)
      d_adv_loss = d_step(config, vocab, model_F, model_D, optimizer_D, batch, temperature)
      his_d_adv_loss.append(d_adv_loss)

    for _ in range(config.iter_F):
      batch = next(batch_iters)
      f_slf_loss, f_cyc_loss, f_adv_loss = f_step(config, vocab, model_F, model_D, optimizer_F, batch, temperature, drop_decay)
      his_f_slf_loss.append(f_slf_loss)
      his_f_cyc_loss.append(f_cyc_loss)
      his_f_adv_loss.append(f_adv_loss)

    global_step += 1
    #writer.add_scalar('rec_loss', rec_loss.item(), global_step)
    #writer.add_scalar('loss', loss.item(), global_step)

    if global_step % config.log_steps == 0:
      avg_d_adv_loss = np.mean(his_d_adv_loss)
      avg_f_slf_loss = np.mean(his_f_slf_loss)
      avg_f_cyc_loss = np.mean(his_f_cyc_loss)
      avg_f_adv_loss = np.mean(his_f_adv_loss)
      
      log_str = '[iter {}] d_adv_loss: {:.4f} ' + \
                'f_slf_loss: {:.4f} f_cyc_loss: {:.4f} ' + \
                'f_adv_loss: {:.4f} temp: {:.4f} drop: {:.4f}' 

      print(log_str.format(
          global_step, avg_d_adv_loss,
          avg_f_slf_loss, avg_f_cyc_loss,avg_f_adv_loss,
          temperature, config.inp_drop_prob * drop_decay
      ))

    if global_step % config.eval_steps == 0:
      his_d_adv_loss = []
      his_f_slf_loss = []
      his_f_cyc_loss = []
      his_f_adv_loss = []

      torch.save(model_F.state_dict(), config.save_folder + '/ckpts' + str(global_step) + '_F.pth')
      torch.save(model_D.state_dict(), config.save_folder + '/ckpts' + str(global_step) + '_D.pth')
      #auto_eval(config, vocab, model_F, test_iters, global_step, temperature)
      #for path, sub_writer in writer.all_writers.items():
      # sub_writer.flush()

#Style Transformer

In [19]:

class StyleTransformer(nn.Module):
    def __init__(self, config, vocab):
        super(StyleTransformer, self).__init__()
        num_styles, num_layers = config.num_styles, config.num_layers
        d_model, max_length = config.d_model, config.max_length
        h, dropout = config.h, config.dropout
        learned_pos_embed = config.learned_pos_embed
        load_pretrained_embed = config.load_pretrained_embed
        
        self.max_length = config.max_length
        self.eos_idx = vocab.stoi['<eos>']
        self.pad_idx = vocab.stoi['<pad>']
        self.style_embed = Embedding(num_styles, d_model)
        self.embed = EmbeddingLayer(
            vocab, d_model, max_length,
            self.pad_idx,
            learned_pos_embed,
            load_pretrained_embed,
        )
        self.sos_token = nn.Parameter(torch.randn(d_model))
        self.encoder = Encoder(num_layers, d_model, len(vocab), h, dropout)
        self.decoder = Decoder(num_layers, d_model, len(vocab), h, dropout)
        
    def forward(self, inp_tokens, gold_tokens, inp_lengths, style,
                generate=False, differentiable_decode=False, temperature=1.0):
        batch_size = inp_tokens.size(0)
        max_enc_len = inp_tokens.size(1)
        my_str = 'the length of max_enc_len is ' + str(max_enc_len)
        print(my_str)

        assert max_enc_len <= self.max_length

        pos_idx = torch.arange(self.max_length).unsqueeze(0).expand((batch_size, -1))
        pos_idx = pos_idx.to(inp_lengths.device)

        src_mask = pos_idx[:, :max_enc_len] >= inp_lengths.unsqueeze(-1)
        src_mask = torch.cat((torch.zeros_like(src_mask[:, :1]), src_mask), 1)
        src_mask = src_mask.view(batch_size, 1, 1, max_enc_len + 1)

        tgt_mask = torch.ones((self.max_length, self.max_length)).to(src_mask.device)
        tgt_mask = (tgt_mask.tril() == 0).view(1, 1, self.max_length, self.max_length)

        style_emb = self.style_embed(style).unsqueeze(1)

        enc_input = torch.cat((style_emb, self.embed(inp_tokens, pos_idx[:, :max_enc_len])), 1)
        memory = self.encoder(enc_input, src_mask)
        
        sos_token = self.sos_token.view(1, 1, -1).expand(batch_size, -1, -1)
        
        if not generate:
            dec_input = gold_tokens[:, :-1]
            max_dec_len = gold_tokens.size(1)
            dec_input_emb = torch.cat((sos_token, self.embed(dec_input, pos_idx[:, :max_dec_len - 1])), 1)
            log_probs = self.decoder(
                dec_input_emb, memory,
                src_mask, tgt_mask[:, :, :max_dec_len, :max_dec_len],
                temperature
            )
        else:
            
            log_probs = []
            next_token = sos_token
            prev_states = None
            
            for k in range(self.max_length):
                log_prob, prev_states = self.decoder.incremental_forward(
                    next_token, memory,
                    src_mask, tgt_mask[:, :, k:k+1, :k+1],
                    temperature,
                    prev_states
                )

                log_probs.append(log_prob)
                
                if differentiable_decode:
                    next_token = self.embed(log_prob.exp(), pos_idx[:, k:k+1])
                else:
                    next_token = self.embed(log_prob.argmax(-1), pos_idx[:, k:k+1])

                #if (pred_tokens == self.eos_idx).max(-1)[0].min(-1)[0].item() == 1:
                #    break

            log_probs = torch.cat(log_probs, 1)
            
            
        return log_probs
    
class Discriminator(nn.Module):
    def __init__(self, config, vocab):
        super(Discriminator, self).__init__()
        num_styles, num_layers = config.num_styles, config.num_layers
        d_model, max_length = config.d_model, config.max_length
        h, dropout = config.h, config.dropout
        learned_pos_embed = config.learned_pos_embed
        load_pretrained_embed = config.load_pretrained_embed
        num_classes = config.num_classes
        
        self.pad_idx = vocab.stoi['<pad>']
        self.style_embed = Embedding(num_styles, d_model)
        self.embed = EmbeddingLayer(
            vocab, d_model, max_length,
            self.pad_idx,
            learned_pos_embed,
            load_pretrained_embed
        )
        self.cls_token = nn.Parameter(torch.randn(d_model))
        self.encoder = Encoder(num_layers, d_model, len(vocab), h, dropout)
        self.classifier = Linear(d_model, num_classes)
    
    def forward(self, inp_tokens, inp_lengths, style=None):
        batch_size = inp_tokens.size(0)
        num_extra_token = 1 if style is None else 2
        max_seq_len = inp_tokens.size(1)

        pos_idx = torch.arange(max_seq_len).unsqueeze(0).expand((batch_size, -1)).to(inp_lengths.device)
        mask = pos_idx >= inp_lengths.unsqueeze(-1)
        for _ in range(num_extra_token):
            mask = torch.cat((torch.zeros_like(mask[:, :1]), mask), 1)
        mask = mask.view(batch_size, 1, 1, max_seq_len + num_extra_token)

        cls_token = self.cls_token.view(1, 1, -1).expand(batch_size, -1, -1)

        enc_input = cls_token
        if style is not None:
            style_emb = self.style_embed(style).unsqueeze(1)
            enc_input = torch.cat((enc_input, style_emb), 1)
            
        enc_input = torch.cat((enc_input, self.embed(inp_tokens, pos_idx)), 1)
        
        encoded_features = self.encoder(enc_input, mask)
        logits = self.classifier(encoded_features[:, 0])

        return F.log_softmax(logits, -1)
        
        
    
class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, vocab_size, h, dropout):
        super(Encoder, self).__init__()
        self.layers = nn.ModuleList([EncoderLayer(d_model, h, dropout) for _ in range(num_layers)])
        self.norm = LayerNorm(d_model)
        
    def forward(self, x, mask):
        y = x

        assert y.size(1) == mask.size(-1)
            
        for layer in self.layers:
            y = layer(y, mask)

        return self.norm(y)
        
class Decoder(nn.Module):
    def __init__(self, num_layers, d_model, vocab_size, h, dropout):
        super(Decoder, self).__init__()
        self.layers = nn.ModuleList([DecoderLayer(d_model, h, dropout) for _ in range(num_layers)])
        self.norm = LayerNorm(d_model)
        self.generator = Generator(d_model, vocab_size)

    def forward(self, x, memory, src_mask, tgt_mask, temperature):
        y = x

        assert y.size(1) == tgt_mask.size(-1)
        
        for layer in self.layers:
            y = layer(y, memory, src_mask, tgt_mask)

        return self.generator(self.norm(y), temperature)
    def incremental_forward(self, x, memory, src_mask, tgt_mask, temperature, prev_states=None):
        y = x

        new_states = []

                                            
        for i, layer in enumerate(self.layers):
            y, new_sub_states = layer.incremental_forward(
                y, memory, src_mask, tgt_mask,
                prev_states[i] if prev_states else None
            )

            new_states.append(new_sub_states)
        
        new_states.append(torch.cat((prev_states[-1], y), 1) if prev_states else y)
        y = self.norm(new_states[-1])[:, -1:]
        
        return self.generator(y, temperature), new_states
    
class Generator(nn.Module):
    def __init__(self, d_model, vocab_size):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x, temperature):
        return F.log_softmax(self.proj(x) / temperature, dim=-1)

class EmbeddingLayer(nn.Module):
    def __init__(self, vocab, d_model, max_length, pad_idx, learned_pos_embed, load_pretrained_embed):
        super(EmbeddingLayer, self).__init__()
        self.token_embed = Embedding(len(vocab), d_model)
        self.pos_embed = Embedding(max_length, d_model)
        self.vocab_size = len(vocab)
        if load_pretrained_embed:
            self.token_embed = nn.Embedding.from_pretrained(vocab.vectors)
            print('embed loaded.')
    def forward(self, x, pos):
        if len(x.size()) == 2:
            y = self.token_embed(x) + self.pos_embed(pos)
        else:
            y = torch.matmul(x, self.token_embed.weight) + self.pos_embed(pos)

        return y
    
class EncoderLayer(nn.Module):
    def __init__(self, d_model, h, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, h, dropout)
        self.pw_ffn = PositionwiseFeedForward(d_model, dropout)
        self.sublayer =  nn.ModuleList([SublayerConnection(d_model, dropout) for _ in range(2)])
        
    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.pw_ffn)


class DecoderLayer(nn.Module):
    def __init__(self, d_model, h, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, h, dropout)
        self.src_attn = MultiHeadAttention(d_model, h, dropout)
        self.pw_ffn = PositionwiseFeedForward(d_model, dropout)
        self.sublayer = nn.ModuleList([SublayerConnection(d_model, dropout) for _ in range(3)])

    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.pw_ffn)

    def incremental_forward(self, x, memory, src_mask, tgt_mask, prev_states=None):
        new_states = []
        m = memory

        x = torch.cat((prev_states[0], x), 1) if prev_states else x
        new_states.append(x)
        x = self.sublayer[0].incremental_forward(x, lambda x: self.self_attn(x[:, -1:], x, x, tgt_mask))
        x = torch.cat((prev_states[1], x), 1) if prev_states else x
        new_states.append(x)
        x = self.sublayer[1].incremental_forward(x, lambda x: self.src_attn(x[:, -1:], m, m, src_mask))
        x = torch.cat((prev_states[2], x), 1) if prev_states else x
        new_states.append(x)
        x = self.sublayer[2].incremental_forward(x, lambda x: self.pw_ffn(x[:, -1:]))
        return x, new_states  
   
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, h, dropout):
        super(MultiHeadAttention, self).__init__()
        assert d_model % h == 0
        self.d_k = d_model // h
        self.h = h
        self.head_projs = nn.ModuleList([nn.Linear(d_model, d_model) for _ in range(3)])
        self.fc = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, query, key, value, mask):
        batch_size = query.size(0)

        query, key, value = [l(x).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)
                             for x, l in zip((query, key, value), self.head_projs)]

        attn_feature, _ = scaled_attention(query, key, value, mask)

        attn_concated = attn_feature.transpose(1, 2).contiguous().view(batch_size, -1, self.h * self.d_k)

        return self.fc(attn_concated)

def scaled_attention(query, key, value, mask):
    d_k = query.size(-1)
    scores = query.matmul(key.transpose(-2, -1)) / math.sqrt(d_k)
    scores.masked_fill_(mask, float('-inf'))
    attn_weight = F.softmax(scores, -1)
    attn_feature = attn_weight.matmul(value)

    return attn_feature, attn_weight
    
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, dropout):
        super(PositionwiseFeedForward, self).__init__()
        self.mlp = nn.Sequential(
            Linear(d_model, 4 * d_model),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            Linear(4 * d_model, d_model),
        )
        
    def forward(self, x):
        return self.mlp(x)

class SublayerConnection(nn.Module):
    def __init__(self, d_model, dropout):
        super(SublayerConnection, self).__init__()
        self.layer_norm = LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, sublayer):
        y = sublayer(self.layer_norm(x))
        return x + self.dropout(y)

    def incremental_forward(self, x, sublayer):
        y = sublayer(self.layer_norm(x))
        return x[:, -1:] + self.dropout(y)
    
def Linear(in_features, out_features, bias=True, uniform=True):
    m = nn.Linear(in_features, out_features, bias)
    if uniform:
        nn.init.xavier_uniform_(m.weight)
    else:
        nn.init.xavier_normal_(m.weight)
    if bias:
        nn.init.constant_(m.bias, 0.)
    return m

def Embedding(num_embeddings, embedding_dim, padding_idx=None):
    m = nn.Embedding(num_embeddings, embedding_dim, padding_idx=padding_idx)
    nn.init.xavier_uniform_(m.weight)
    nn.init.constant_(m.weight[padding_idx], 0)
    return m

def LayerNorm(embedding_dim, eps=1e-6):
    m = nn.LayerNorm(embedding_dim, eps)
    return m

#Start???

In [20]:
train_iters, dev_iters, test_iters, vocab = load_dataset(config)

In [ ]:
#index2word = vocab.itos

In [ ]:
#vocab.stoi['<unk>']

In [ ]:
#index2word[pad_idx]

In [21]:
print('Vocab size:', len(vocab))
model_F = StyleTransformer(config, vocab).to(config.device)
model_D = Discriminator(config, vocab).to(config.device)
print(config.discriminator_method)

Vocab size: 34685
Multi


In [1]:
train(config, vocab, model_F, model_D, train_iters, dev_iters, test_iters)

NameError: ignored

#sAVED stuff


In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [23]:
model_F.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/style_transfer/training_files/save/Aug08180829/ckpts1175_F.pth',map_location=device))
model_F.eval()

StyleTransformer(
  (style_embed): Embedding(2, 256)
  (embed): EmbeddingLayer(
    (token_embed): Embedding(34685, 256)
    (pos_embed): Embedding(400, 256)
  )
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadAttention(
          (head_projs): ModuleList(
            (0): Linear(in_features=256, out_features=256, bias=True)
            (1): Linear(in_features=256, out_features=256, bias=True)
            (2): Linear(in_features=256, out_features=256, bias=True)
          )
          (fc): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0, inplace=False)
        )
        (pw_ffn): PositionwiseFeedForward(
          (mlp): Sequential(
            (0): Linear(in_features=256, out_features=1024, bias=True)
            (1): ReLU(inplace=True)
            (2): Dropout(p=0, inplace=False)
            (3): Linear(in_features=1024, out_features=256, bias=True)
          )
        )
        (sublayer): M

In [24]:
model_D.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/style_transfer/training_files/save/Aug08180829/ckpts1175_D.pth',map_location=device))
model_D.eval()

Discriminator(
  (style_embed): Embedding(2, 256)
  (embed): EmbeddingLayer(
    (token_embed): Embedding(34685, 256)
    (pos_embed): Embedding(400, 256)
  )
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadAttention(
          (head_projs): ModuleList(
            (0): Linear(in_features=256, out_features=256, bias=True)
            (1): Linear(in_features=256, out_features=256, bias=True)
            (2): Linear(in_features=256, out_features=256, bias=True)
          )
          (fc): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0, inplace=False)
        )
        (pw_ffn): PositionwiseFeedForward(
          (mlp): Sequential(
            (0): Linear(in_features=256, out_features=1024, bias=True)
            (1): ReLU(inplace=True)
            (2): Dropout(p=0, inplace=False)
            (3): Linear(in_features=1024, out_features=256, bias=True)
          )
        )
        (sublayer): Modu

#Save predictions

In [ ]:
gold_text = []
raw_output = []
rev_output = []

In [ ]:
#probably should have these in the loop
inp_styles
raw_styles = torch.full_like(inp_tokens[:, 0], 0)
rev_styles = 1 - raw_styles
rev_styles

In [32]:
loss_fn = nn.NLLLoss(reduction = 'none')

In [33]:
print(len(vocab))
pad_idx = vocab.stoi['pad']
eos_idx = vocab.stoi['eos']
for batch in train_iters:
    inp_tokens, inp_lengths, inp_styles=batch_preprocess(batch,pad_idx,eos_idx)
    rev_styles = 1 - inp_styles
     with torch.no_grad():
         raw_log_probs = model_F(
             inp_tokens,
             None,
             inp_lengths,
             raw_styles,
             generate=True,
             differentiable_decode=False,
            temperature=1,
  )
            
  with torch.no_grad():
      rev_log_probs = model_F(
          inp_tokens, 
          None,
          inp_lengths,
          rev_styles,
          generate=True,
          differentiable_decode=False,
          temperature=1,
      )
                
gold_text += tensor2text(vocab, inp_tokens.cpu())
raw_output += tensor2text(vocab, raw_log_probs.argmax(-1).cpu())
rev_output += tensor2text(vocab, rev_log_probs.argmax(-1).cpu())

34685


NameError: ignored

In [40]:
gold_text

['Variable numbers of immature white blood cells and platelets also may be present in the blood .',
 'Blunt trauma may cause ecchymosis ( eg , the transverse , linear ecchymosis termed seat belt sign ) , but this finding has poor sensitivity and specificity .',
 'For such people , doctors often do the following : A complete blood count ( including the number and proportion of different types of white blood cells ) ; A chest x-ray ; Urinalysis .',
 'If adrenal crisis is suspected , confirmation of Addison disease by ACTH stimulation testing is deferred until the patient has recovered .',
 'However , if multiple mononeuropathy involves many nerves , it may be difficult to distinguish from polyneuropathy .',
 'These variants may cause the gene to malfunction , possibly resulting in increased production of beta- amyloid .',
 'Doctors then do a physical examination .',
 'Flexible spending accounts ( ) are offered by some employers .',
 'Narrowing of the airways is often caused by abnormal s

In [45]:
rev_output

["The most common cause of the person 's body and may be used in the child 's body .",
 'The most common cause of the person may be used , and may be used to be used .',
 'The following : , and the ( eg , and can be used for the ) .',
 'The diagnosis is not by the first or by the first or by the first .',
 'The diagnosis , may be done to be done to be done to be done to be done .',
 "The diagnosis of the child may be used in the child 's body , but may be used to be used .",
 "The most common cause of a person 's evaluation .",
 '( See also Overview of .',
 'The diagnosis of the diagnosis of the first disorder is not used to be used to be used to be used to the first .',
 "The diagnosis of the diagnosis are not used for the child 's body ( eg , but are not used ) .",
 'The diagnosis , and the first ( eg , and the first ) is not used in the first .',
 'The diagnosis can be used to be used for the first with the first disorder .',
 'and ( eg , and ) and .',
 'The diagnosis of the diagnos

#Save results


In [56]:
with open("/content/drive/My Drive/Colab Notebooks/style_transfer/training_files/gold_test_trial1.txt", "w") as output:
  for line in gold_text:
    output.write(line + '\n')

In [57]:
with open("/content/drive/My Drive/Colab Notebooks/style_transfer/training_files/raw_output_trial1.txt", "w") as output:
  for line in raw_output:
    output.write(line + '\n')

In [58]:
with open("/content/drive/My Drive/Colab Notebooks/style_transfer/training_files/rev_output_trial1.txt", "w") as output:
  for line in rev_output:
    output.write(line + '\n')

#Evaluation BLEU score

In [50]:
  
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu


In [52]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [47]:
def nltk_bleu(texts_origin, text_transfered):
  texts_origin = [word_tokenize(text_origin.lower().strip()) for text_origin in texts_origin]
  text_transfered = word_tokenize(text_transfered.lower().strip())
  return sentence_bleu(texts_origin, text_transfered) * 100

In [59]:
bleu_scores_raw = []
bleu_scores_rev = []
for i in range(len(gold_text)):
  bleu_scores_raw.append(nltk_bleu(gold_text[i],raw_output[i]))
  bleu_scores_rev.append(nltk_bleu(gold_text[i],rev_output[i]))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [63]:

import statistics
statistics.mean(bleu_scores_raw)

57.07954434495195

In [64]:

import statistics
statistics.mean(bleu_scores_rev)

56.821192493168574

#Save BLEUS

In [66]:
with open("/content/drive/My Drive/Colab Notebooks/style_transfer/training_files/bleus_rev.txt", "w") as output:
  for line in bleu_scores_rev:
    output.write(str(line) + '\n')

with open("/content/drive/My Drive/Colab Notebooks/style_transfer/training_files/bleus_raw.txt", "w") as output:
  for line in bleu_scores_raw:
    output.write(str(line) + '\n')

In [46]:
# n-gram individual BLEU
from nltk.translate.bleu_score import sentence_bleu
#reference = [['this', 'is', 'a', 'test']]
#candidate = ['this', 'is', 'a', 'test']
#print('Individual 1-gram: %f' % sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
#print('Individual 2-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 1, 0, 0)))
#print('Individual 3-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 1, 0)))
print('Individual 4-gram: %f' % sentence_bleu(gold_text[10], raw_output[10], weights=(0, 0, 0, 1)))

Individual 4-gram: 1.000000


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [32]:
 with torch.no_grad():
  raw_log_probs = model_F(
      inp_tokens,
      None,
      inp_lengths,
      raw_styles,
      generate=True,
      differentiable_decode=False,
      temperature=1,
  )
            
with torch.no_grad():
    rev_log_probs = model_F(
        inp_tokens, 
        None,
        inp_lengths,
        rev_styles,
        generate=True,
        differentiable_decode=False,
        temperature=1,
    )
                
gold_text += tensor2text(vocab, inp_tokens.cpu())
raw_output += tensor2text(vocab, raw_log_probs.argmax(-1).cpu())
rev_output += tensor2text(vocab, rev_log_probs.argmax(-1).cpu())

the length of max_enc_len is 29
the length of max_enc_len is 29


In [36]:
gold_text

['Variable numbers of immature white blood cells and platelets also may be present in the blood .',
 'Blunt trauma may cause ecchymosis ( eg , the transverse , linear ecchymosis termed seat belt sign ) , but this finding has poor sensitivity and specificity .']

In [54]:
inp_styles

tensor([1, 0], device='cuda:0')

In [55]:
rev_styles = 1 - inp_styles

In [56]:
rev_styles

tensor([0, 1], device='cuda:0')

In [57]:
inp_tokens

tensor([[  95,    3,  100,   51,   81,   26, 3792,    5,   87, 2740,  142, 3458,
            4,    2, 5912, 5912, 5912, 5912, 5912],
        [3466, 1581,   22, 4343,   44,   13,  552,  127,   14,   34,   17,   13,
          118,  134,   15,   38, 1100,    4,    2]], device='cuda:0')

In [41]:
raw_output = tensor2text(vocab, raw_gen_log_probs.argmax(-1).cpu())

In [42]:
raw_output

['The of in the disorder and the and in the body .',
 'The of the are of a of the disease of the is .']

In [ ]:
rev_output = tensor2text(vocab, rev_gen_log_probs.argmax(-1).cpu())

#Cache clean

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 1            |        cudaMalloc retries: 1         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   14822 MB |   14822 MB |    1665 GB |    1650 GB |\n|       from large pool |   12145 MB |   12145 MB |    1582 GB |    1570 GB |\n|       from small pool |    2677 MB |    2679 MB |      82 GB |      80 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |   14822 MB |   14822 MB |    1665 GB |    1650 GB |\n|       from large pool |   12145 MB |   12145 MB |